In [91]:
import pandas as pd 

In [92]:
data = pd.read_csv('TestDataSet/titanic_train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [93]:
input_feature = data.drop(['PassengerId','Survived','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns')
input_feature.head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


### Cleaning the data

In [94]:
input_feature['Age'].fillna(input_feature['Age'].median(), inplace=True)
input_feature.head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


### Encoding [sex] column

In [95]:
from sklearn.preprocessing import LabelEncoder
LE_sex = LabelEncoder()

In [96]:
input_feature['sex_n'] = LE_sex.fit_transform(input_feature['Sex'])
input_feature.head()

,Pclass,Sex,Age,Fare,sex_n
0,3,male,22.0,7.2500,1
1,1,female,38.0,71.2833,0
2,3,female,26.0,7.9250,0
3,1,female,35.0,53.1000,0
4,3,male,35.0,8.0500,1


## Dropping the [sex] column

In [97]:
input_feature = input_feature.drop(['Sex'],axis='columns')
input_feature.head()

,Pclass,Age,Fare,sex_n
0,3,22.0,7.2500,1
1,1,38.0,71.2833,0
2,3,26.0,7.9250,0
3,1,35.0,53.1000,0
4,3,35.0,8.0500,1


### Separating the target feature

In [98]:
target = data['Survived']
target.info()

<class 'pandas.core.series.Series'>
RangeIndex: 891 entries, 0 to 890
Series name: Survived
Non-Null Count  Dtype
--------------  -----
891 non-null    int64
dtypes: int64(1)
memory usage: 7.1 KB


### Test Train Split

In [99]:
from sklearn.model_selection import train_test_split
X_train,X_test, Y_train, Y_test = train_test_split(input_feature,target,test_size=0.1,stratify=target,random_state=50)
# test size = 0.1 means , training will be done on 90% of the dataset

### Training The Model

In [100]:
from sklearn import tree
model = tree.DecisionTreeClassifier()

In [101]:
model.fit(X_train,Y_train)

DecisionTreeClassifier()

In [102]:
model.score(X_test,Y_test)

0.8888888888888888

### Now , loading the test data set

In [103]:
testing_df = pd.read_csv('TestDataSet/Titanic_test.csv')
testing_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### Taking only the input feature and cleaning up the dataset

In [104]:
final_input = testing_df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns')
final_input['Age'].fillna(final_input['Age'].mean(), inplace=True)
final_input['sex_n'] = LE_sex.fit_transform(final_input['Sex'])
final_input = final_input.drop(['Sex'],axis='columns')
final_input.head()

,Pclass,Age,Fare,sex_n
0,3,34.5,7.8292,1
1,3,47.0,7.0000,0
2,2,62.0,9.6875,1
3,3,27.0,8.6625,1
4,3,22.0,12.2875,0


In [105]:
prediction_result = model.predict(final_input)

In [106]:
prediction_result.size

418

In [107]:
pass_id = testing_df['PassengerId']
pass_id.size

418

In [108]:
df = pd.DataFrame({
    "PassengerId" : pass_id.values,
    "Survived" : prediction_result
})


In [109]:
df.to_csv('submission2.csv',index=False)